In [1]:
#.py with dataframe operations

# Standard Imports
import pandas as pd
import numpy as np
import json
import jellyfish


In [4]:

# Import data
datos_col = pd.read_csv('data/homicidios_policia.csv',
                        sep=';',
                        parse_dates=["FECHA HECHO"],
                        dayfirst=True
                       );

cod_dane= pd.read_csv('data/cod_dane.csv')
pobl = pd.read_csv('Data/poblacion_municipios.csv')


In [6]:

datos_col.columns = ['departamento','municipio','codigo','arma','fecha','genero', 'edad_grupo', 'cantidad']

# add population
datos_col['codigo'] = datos_col['codigo'].astype(str).str[:-3].astype(int)
cod_dane['Código Municipio']  = cod_dane['Código Municipio'].str.replace(',', '')
cod_dane['Código Municipio'] = cod_dane['Código Municipio'].astype(int)


In [8]:
cod_dane.head()

,Código Departamento,Código Municipio,Código Centro Poblado,Nombre Departamento,Nombre Municipio,Nombre Centro Poblado,Tipo Centro Poblado,Ubicación,Longitud,Latitud,Distrito,Tipo de Municipio,Area Metropolitana
0,5,5001,"5,001,000",ANTIOQUIA,MEDELLÍN,MEDELLÍN,CABECERA MUNICIPAL (CM),"(6.24858636743, -75.5760017134)",-75.576002,6.248586,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ
1,5,5001,"5,001,001",ANTIOQUIA,MEDELLÍN,PALMITAS,CABECERA CORREGIMIENTO MUNICIPAL (C),"(6.3439224059, -75.6899859872)",-75.689986,6.343922,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ
2,5,5001,"5,001,004",ANTIOQUIA,MEDELLÍN,SANTA ELENA,CABECERA CORREGIMIENTO MUNICIPAL (C),"(6.20967456778, -75.5003166982)",-75.500317,6.209675,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ
3,5,5001,"5,001,009",ANTIOQUIA,MEDELLÍN,ALTAVISTA,CABECERA CORREGIMIENTO MUNICIPAL (C),"(6.22248273057, -75.6299441751)",-75.629944,6.222483,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ
4,5,5001,"5,001,010",ANTIOQUIA,MEDELLÍN,AGUAS FRÍAS,CABECERA CORREGIMIENTO MUNICIPAL (C),"(6.2333355874, -75.6339460169)",-75.633946,6.233336,NaN,MUNICIPIO,AREA METROPOLITANA DEL VALLE DE ABURRÁ


In [9]:

pobl = pobl.merge(cod_dane[['Código Municipio','Longitud','Latitud']], on = 'Código Municipio', how = 'left')

In [10]:
pobl.head()

,DP,DPNOM,Código Municipio,MPIO,AÑO,ÁREA GEOGRÁFICA,Pobl,Longitud,Latitud
0,5,Antioquia,5001,Medellín,2022,Total,"2,612,958",-75.576002,6.248586
1,5,Antioquia,5001,Medellín,2022,Total,"2,612,958",-75.689986,6.343922
2,5,Antioquia,5001,Medellín,2022,Total,"2,612,958",-75.500317,6.209675
3,5,Antioquia,5001,Medellín,2022,Total,"2,612,958",-75.629944,6.222483
4,5,Antioquia,5001,Medellín,2022,Total,"2,612,958",-75.633946,6.233336


In [11]:

pobl['Pobl']  = pobl['Pobl'].str.replace(',', '').astype(int)
pobl.head()


,DP,DPNOM,Código Municipio,MPIO,AÑO,ÁREA GEOGRÁFICA,Pobl,Longitud,Latitud
0,5,Antioquia,5001,Medellín,2022,Total,2612958,-75.576002,6.248586
1,5,Antioquia,5001,Medellín,2022,Total,2612958,-75.689986,6.343922
2,5,Antioquia,5001,Medellín,2022,Total,2612958,-75.500317,6.209675
3,5,Antioquia,5001,Medellín,2022,Total,2612958,-75.629944,6.222483
4,5,Antioquia,5001,Medellín,2022,Total,2612958,-75.633946,6.233336


In [ ]:

#calculate total homicides
total = datos_col['cantidad'].sum()

#clearing cod_dane
cod_dane['Código Centro Poblado']=cod_dane['Código Centro Poblado'].apply(lambda x: x.replace(',',''))
cod_dane['Código Centro Poblado'] = cod_dane['Código Centro Poblado'].astype(int)

#clearing dotos_col
df = datos_col.copy()
keep_values =  ["ANTIOQUIA","VALLE","CUNDINAMARCA"]
df = df[df["departamento"].isin(keep_values)]
df['edad_grupo'].replace(' ', np.nan, inplace=True)
df = df.dropna()

dict_values =  {
"ARMA BLANCA / CORTOPUNZANTE":"ARMA BLANCA",
"ARMA DE FUEGO":"ARMA DE FUEGO",
"CORTANTES":"ARMA BLANCA",
"PUNZANTES":"ARMA BLANCA",
"CONTUNDENTES":"OTRAS ARMAS",
"ARTEFACTO EXPLOSIVO/CARGA DINAMITA":"OTRAS ARMAS",
"MINA ANTIPERSONA":"OTRAS ARMAS",
"CUERDA/SOGA/CADENA":"OTRAS ARMAS",
"COMBUSTIBLE":"OTRAS ARMAS",
"BOLSA PLASTICA":"OTRAS ARMAS",
"MOTO BOMBA":"OTRAS ARMAS",
"GRANADA DE MANO":"OTRAS ARMAS",
"PAQUETE BOMBA":"OTRAS ARMAS",
"SUSTANCIAS TOXICAS":"OTRAS ARMAS",
"SIN EMPLEO DE ARMAS":"OTRAS ARMAS",
"JERINGA":"OTRAS ARMAS",
"CARRO BOMBA":"OTRAS ARMAS",
"NO REPORTADO":"OTRAS ARMAS",
"PERSONA BOMBA":"OTRAS ARMAS",
"CINTAS/CINTURON":"OTRAS ARMAS",
"ESCOPOLAMINA":"OTRAS ARMAS",
"ALMOHADA":"OTRAS ARMAS",
"CILINDRO BOMBA":"OTRAS ARMAS",
"ARTEFACTO INCENDIARIO":"OTRAS ARMAS",
"VENENO":"OTRAS ARMAS",
"ROCKET":"OTRAS ARMAS",
"QUIMICOS":"OTRAS ARMAS",
"OLLA BOMBA":"OTRAS ARMAS",
"GASES":"OTRAS ARMAS",
"NO REPORTADA":"OTRAS ARMAS",
"GRANADA DE MORTERO":"OTRAS ARMAS",
"CASA BOMBA":"OTRAS ARMAS",
"MEDICAMENTOS":"OTRAS ARMAS",
"ACIDO":"OTRAS ARMAS",
"POLVORA(FUEGOS PIROTECNICOS)":"OTRAS ARMAS",
"PRENDAS DE VESTIR":"OTRAS ARMAS",
"LIQUIDOS":"OTRAS ARMAS"
}

#df base
df = df.replace({"arma": dict_values})

df['ano'] = df['fecha'].dt.year
df["dia"] = df["fecha"].dt.day_name()
df = df.reindex(columns=['fecha','ano','dia','departamento','municipio','codigo', 'arma', 'genero', 'edad_grupo', 'cantidad'])

#for chart per year
df_yearly = df.groupby("ano")['cantidad'].sum()
df_yearly = df_yearly.to_frame().reset_index()

#for chart per month
df['mes'] = df['fecha'].dt.month
df_monthly = df.groupby("mes")['cantidad'].sum()
df_monthly = df_monthly.to_frame().reset_index()

#for chart per day
df_daily = df.groupby("dia")['cantidad'].sum()
df_daily = df_daily.to_frame().reset_index()

#for chart serie times
df['ano_mes'] = df['fecha'].dt.to_period('M').astype(dtype=str)
df_year_month = df.groupby("ano_mes")['cantidad'].sum()
df_year_month = df_year_month.to_frame().reset_index()

#for chart range of age
df_age = df.groupby("edad_grupo")['cantidad'].sum()
df_age = df_age.to_frame().reset_index()

#for chart per gender
df_gen = df.groupby("genero")['cantidad'].sum()
df_gen = df_gen.to_frame().reset_index()

#for chart per kind of weapons
df_weapon = df.groupby("arma")['cantidad'].sum()
df_weapon = df_weapon.to_frame().reset_index()

#for chart box plot per departamento
df['week'] = df['fecha'].dt.to_period('M')
df_departamento = df.groupby(["departamento","week"])['cantidad'].sum()
df_departamento = df_departamento.to_frame().reset_index()

#for charta box plot per municipio
df['week'] = df['fecha'].dt.to_period('M')
df_municipio = df.groupby(["municipio","week","departamento"])['cantidad'].sum()
df_municipio = df_municipio.to_frame().reset_index()
df_municipio = df_municipio.sort_values(by='cantidad', ascending=False)
df_municipio = df_municipio[df_municipio['cantidad'] > 20]

#for chart departamento x year
df_dpto_year = df.groupby(["ano","departamento"])['cantidad'].sum()
df_dpto_year = df_dpto_year.reset_index()

#for chart departamento x month
df_dpto_month = df.groupby(["mes","departamento"])['cantidad'].sum()
df_dpto_month = df_dpto_month.reset_index()

#for chart departamento x day
df_dpto_day = df.groupby(["dia","departamento"])['cantidad'].sum()
df_dpto_day = df_dpto_day.reset_index()

#merge df with cod dane
df_hom = pd.merge(df, cod_dane[['Código Centro Poblado','Tipo Centro Poblado', 'Longitud', 'Latitud', 'Pobl']],
                  left_on='codigo', 
                  right_on='Código Centro Poblado', 
                  how='left')

df_hom = df_hom.drop(columns=["Código Centro Poblado"])

#merge df with cod dane
df_hom2 = pd.merge(df, cod_dane[['Código Municipio','Tipo Centro Poblado', 'Longitud', 'Latitud', 'Pobl']],
                  left_on='codigo', 
                  right_on='Código Municipio', 
                  how='left')

df_hom2 = df_hom2.drop(columns=["Código Municipio"])

""""
# change names of municipalies
cod_dane['Código Centro Poblado'] = cod_dane['Código Centro Poblado'].astype(int)
df_hom['codigo'] = df_hom['codigo'].astype(int)
df_hom['municipio'] = df_hom.municipio.str.replace('(CT)', '')
df_hom['municipio'] = df_hom.municipio.str.replace('[(+*)]', '')
df_hom['municipio'] = df_hom.municipio.str.replace('\s', '')

# load geojson municipalies
f = open("data/MunicipiosEdit.json", encoding="utf8")
geojson = json.load(f)
json_names = []
for loc in geojson['features']:
    loc['id'] = loc['properties']['MPIO_CNMBR']
    json_names.append(loc['id'])

# change names in df
df_names = list(df_hom.municipio.unique())
dict_mun ={}
dist_list = []
min_dist = 1000
match = ''
for name in df_names:
    for name2 in json_names:
        dist = jellyfish.levenshtein_distance(name, name2)
        if dist < min_dist:
            min_dist = dist
            match = name2
        
    dist_list.append(min_dist)
    dict_mun[name] = match
    min_dist = 1000
    
df_hom['municipio'] = df_hom.municipio.replace(dict_mun)
"""